This notebook is to contain notes on different models, leading up to a decision on which model to use in production.

This notebook incorporates the dataloader defined in https://github.com/axelth/us8kdata , and will also incorporate the feature extraction pipeline prototyped in notebooks/.... and implemented in hasp/{features.py, pipeline.py} etc.

imports for individual models are performed in their respective section to make it easier to see which code to transfer to the library after model selection

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImPipeline
from sklearn.metrics import classification_report, confusion_matrix

from us8kdata.loader import UrbanSound8K
from hasp.feature_pipeline import make_feature_pipeline
from hasp.feature_pipeline import make_oversampled_feature_pipeline
from hasp.augmenting_oversampler import AugmentingRandomOversampler as OverSampler
from hasp.util import combine_classes

import mlflow
from hasp.mlflow_logging import MLFlowExperiment

In [48]:
%%time
data = UrbanSound8K('../hasp/data')
train_fold = [samples for samples in data.fold_audio_generator(fold=[1, 2, 3, 6, 7, 8, 9, 10],
                                                              classID=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])]
val_fold = [samples for samples in data.fold_audio_generator(fold=[4],
                                                            classID=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])]
test_fold = [samples for samples in data.fold_audio_generator(fold=[5],
                                                             classID=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])]
train_y = data.filter_metadata(fold=[1, 2, 3, 6, 7, 8, 9, 10],
                              classID=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]).classID
val_y = data.filter_metadata(fold=[4],
                            classID=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]).classID
test_y = data.filter_metadata(fold=[5],
                             classID=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]).classID

CPU times: user 332 ms, sys: 634 ms, total: 966 ms
Wall time: 3.98 s


In [33]:
data.metadata[['classID', 'class']]

,classID,class
0,3,dog_bark
1,2,children_playing
2,2,children_playing
3,2,children_playing
4,2,children_playing
...,...,...
8727,1,car_horn
8728,1,car_horn
8729,1,car_horn
8730,1,car_horn


In [6]:
feature_pipe = make_feature_pipeline()
feature_pipe

Pipeline(steps=[('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x1034ed670>,
                                     kw_args={'fmax': None, 'fmin': 0.0,
                                              'hop_length': 128, 'n_fft': 512,
                                              'n_mfcc': 20, 'sr': 16000})),
                ('scaler', StandardScaler())])

In [7]:
train_pre = feature_pipe.fit_transform(train_fold)
val_pre = feature_pipe.transform(val_fold)
#test_pre = feature_pipe.transform(test_fold)

# Single sample classification
Model architectures for treating each audio file as a single example and classifying it in one go.

## Logistic Regression (baseline)

### LASSO

## KNN

## SVC

In [6]:
from sklearn.svm import SVC


In [ ]:
model_svc = SVC(C=4)
model_svc.fit(train_pre, train_y)

In [18]:
#val_pre.shape
svc_val_prediction = model_svc.predict(val_pre)
print(classification_report(
    combine_classes([1, 8], val_y),
    combine_classes([1, 8], svc_val_prediction)
))
print(classification_report(
     val_y, 
    svc_val_prediction))
pd.DataFrame(confusion_matrix(val_y, svc_val_prediction), 
             index=[0,1,2,3,4,5,6,7,8,9], 
             columns=[0,1,2,3,4,5,6,7,8,9])

              precision    recall  f1-score   support

           1       0.57      0.42      0.49        59
           7       0.92      0.96      0.94       765
           8       0.91      0.82      0.86       166

    accuracy                           0.90       990
   macro avg       0.80      0.73      0.76       990
weighted avg       0.90      0.90      0.90       990

              precision    recall  f1-score   support

           0       0.52      0.43      0.47       100
           1       0.57      0.42      0.49        59
           2       0.47      0.42      0.44       100
           3       0.41      0.75      0.53       100
           4       0.51      0.63      0.56       100
           5       0.31      0.22      0.26       107
           6       0.50      0.61      0.55        38
           7       0.19      0.04      0.07       120
           8       0.91      0.82      0.86       166
           9       0.28      0.47      0.35       100

    accuracy           

,0,1,2,3,4,5,6,7,8,9
0,43,0,7,12,24,0,0,3,0,11
1,0,25,1,6,8,3,1,6,1,8
2,10,4,42,17,2,0,5,2,0,18
3,0,0,11,75,0,0,4,0,6,4
4,6,3,1,3,63,9,1,6,0,8
5,4,1,1,20,0,24,12,0,5,40
6,0,0,2,4,2,0,23,4,0,3
7,13,9,7,0,22,38,0,5,0,26
8,0,0,5,25,0,0,0,0,136,0
9,7,2,13,23,3,3,0,1,1,47


In [30]:
%%time
svc_ovsamp_pipeline = make_oversampled_feature_pipeline()
svc_ovsamp_pipeline.set_params(
**{'estimator': SVC(C=4),
 'mean_mfcc__kw_args': {'sr': 16000,'n_mfcc': 16,
  'n_fft': 256,'hop_length': 128,
  'fmin': 0.0, 'fmax': 5000, 'n_mels': 98},
  'over_sampler': OverSampler(augment_method='AddGaussianNoise')}
)
svc_ovsamp_pipeline.fit(train_fold, train_y)

Use pipe.set_params(over_sampler=.., estimator=..) to finish the pipeline
CPU times: user 2min 18s, sys: 41.7 s, total: 2min 59s
Wall time: 28.5 s


Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x28019cee0>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x148b6d820>,
                                     kw_args={'fmax': 5000, 'fmin': 0.0,
                                              'hop_length': 128, 'n_fft': 256,
                                              'n_mels': 98, 'n_mfcc': 16,
                                              'sr': 16000})),
                ('scaler', StandardScaler()), ('estimator', SVC(C=4))])

In [44]:
svc_ovsamp_pipeline[:-1].

Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x28019cee0>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x148b6d820>,
                                     kw_args={'fmax': 5000, 'fmin': 0.0,
                                              'hop_length': 128, 'n_fft': 256,
                                              'n_mels': 98, 'n_mfcc': 16,
                                              'sr': 16000})),
                ('scaler', StandardScaler())])

In [33]:
svc_ovsamp_val_pred = svc_ovsamp_pipeline.predict(val_fold)
print(classification_report(
    combine_classes([1, 8], val_y),
    combine_classes([1, 8], svc_ovsamp_val_pred)
))
print(classification_report(
     val_y, 
    svc_ovsamp_val_pred))
labels = data.metadata[['classID', 'class']].drop_duplicates().sort_values('classID')['class'].to_list()
pd.DataFrame(confusion_matrix(val_y, svc_ovsamp_val_pred),
             index=labels,#[0,1,2,3,4,5,6,7,8, 9], 
             columns=labels)#[0,1,2,3,4,5,6,7,8, 9])

              precision    recall  f1-score   support

           1       0.51      0.46      0.48        59
           7       0.92      0.93      0.93       765
           8       0.80      0.80      0.80       166

    accuracy                           0.88       990
   macro avg       0.74      0.73      0.74       990
weighted avg       0.88      0.88      0.88       990

              precision    recall  f1-score   support

           0       0.51      0.50      0.50       100
           1       0.51      0.46      0.48        59
           2       0.55      0.41      0.47       100
           3       0.53      0.77      0.63       100
           4       0.45      0.55      0.49       100
           5       0.35      0.24      0.29       107
           6       0.77      0.53      0.62        38
           7       0.14      0.08      0.10       120
           8       0.80      0.80      0.80       166
           9       0.28      0.44      0.34       100

    accuracy           

,air_conditioner,car_horn,children_playing,dog_bark,drilling,engine_idling,gun_shot,jackhammer,siren,street_music
air_conditioner,50,4,6,2,11,0,0,13,10,4
car_horn,0,27,0,2,7,3,0,7,2,11
children_playing,12,0,41,13,12,0,0,4,0,18
dog_bark,1,2,8,77,2,0,5,0,4,1
drilling,3,2,4,1,55,6,0,26,0,3
engine_idling,9,0,1,4,0,26,0,6,16,45
gun_shot,0,5,1,4,2,0,20,4,0,2
jackhammer,10,6,1,2,31,32,0,10,0,28
siren,0,3,1,26,0,2,1,0,133,0
street_music,14,4,12,13,3,5,0,3,2,44


In [13]:
data.metadata[['classID', 'class']].drop_duplicates().sort_values('class')

,classID,class
22,0,air_conditioner
9,1,car_horn
1,2,children_playing
0,3,dog_bark
196,4,drilling
122,5,engine_idling
106,6,gun_shot
171,7,jackhammer
114,8,siren
94,9,street_music


### try mlflow logging

In [49]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support

In [62]:
%%time
svc_experiment = MLFlowExperiment("axelth", 'svc_oversample',version="0.1")
binary_train = True
target_classes = [1, 3, 8]
for delta1 in (False, True):
    for delta2 in (False, True):
        svc_experiment.mlflow_create_run()

        tmp_pipeline = make_oversampled_feature_pipeline()
        tmp_pipeline.set_params(
            **{'estimator': SVC(C=4),
             'mean_mfcc__kw_args': {'sr': 16000,'n_mfcc': 16,
              'n_fft': 256,'hop_length': 128,
              'fmin': 0.0, 'fmax': 5000, 'n_mels': 98, 'delta1': delta1, 'delta2': delta2},
              'over_sampler': OverSampler(augment_method='AddGaussianNoise')}
        )
        for paramkey in ['estimator', 'mean_mfcc__kw_args', 'over_sampler']:
            paramval = tmp_pipeline.get_params()[paramkey]
            if type(paramval) == dict:
                svc_experiment.log_kwargs_param('mean_mfcc', paramval)
            else:
                svc_experiment.mlflow_log_param(paramkey, paramval)
        svc_experiment.mlflow_log_param('binary_train', binary_train)
        if binary_train:
            tmp_pipeline.fit(train_fold, combine_classes(target_classes, train_y, binary=binary_train))
            
        tmp_val_pred = tmp_pipeline.predict(val_fold)
        
        if not binary_train:
            tmp_val_pred = combine_classes(target_classes, tmp_val_pred, binary=True)
        
        tmp_eval = precision_recall_fscore_support(
            tmp_val_pred,
            combine_classes(target_classes, val_y, binary=True),
            average='binary'
        )
        print(tmp_eval)
        svc_experiment.mlflow_log_metric('fscore', tmp_eval[2])
        svc_experiment.mlflow_client.set_terminated(svc_experiment.mlflow_run.info.run_id)
        

Use pipe.set_params(over_sampler=.., estimator=..) to finish the pipeline
(0.8523076923076923, 0.6907730673316709, 0.7630853994490359, None)
Use pipe.set_params(over_sampler=.., estimator=..) to finish the pipeline
(0.8523076923076923, 0.6674698795180722, 0.7486486486486487, None)
Use pipe.set_params(over_sampler=.., estimator=..) to finish the pipeline
(0.8307692307692308, 0.6428571428571429, 0.7248322147651007, None)
Use pipe.set_params(over_sampler=.., estimator=..) to finish the pipeline
(0.8307692307692308, 0.645933014354067, 0.7267833109017496, None)
CPU times: user 17min 53s, sys: 3min 49s, total: 21min 43s
Wall time: 5min 7s


In [57]:
np.where(np.isin(tmp_val_pred, [1,3,8]), 1, 0)
# tmp_val_pred

array([0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

## RandomForest

## XGBoost

In [37]:
from xgboost import XGBClassifier
#model_xgb = XGBClassifier()
#model_xgb.fit(train_pre, train_y)

In [19]:
xgb_val_prediction = model_xgb.predict(val_pre)
print(classification_report(
    combine_classes([1, 8], val_y),
    combine_classes([1, 8], xgb_val_prediction)
))
print(classification_report(
     val_y, 
    xgb_val_prediction))
pd.DataFrame(confusion_matrix(val_y, xgb_val_prediction),
             index=[0,1,2,3,4,5,6,7,8,9], 
             columns=[0,1,2,3,4,5,6,7,8,9])

              precision    recall  f1-score   support

           1       0.67      0.37      0.48        59
           7       0.92      0.94      0.93       765
           8       0.80      0.83      0.81       166

    accuracy                           0.89       990
   macro avg       0.80      0.71      0.74       990
weighted avg       0.88      0.89      0.88       990

              precision    recall  f1-score   support

           0       0.20      0.08      0.11       100
           1       0.67      0.37      0.48        59
           2       0.42      0.41      0.41       100
           3       0.46      0.75      0.57       100
           4       0.47      0.55      0.50       100
           5       0.35      0.26      0.30       107
           6       0.47      0.66      0.55        38
           7       0.28      0.10      0.15       120
           8       0.80      0.83      0.81       166
           9       0.26      0.49      0.34       100

    accuracy           

,0,1,2,3,4,5,6,7,8,9
0,8,3,22,7,15,9,0,2,8,26
1,2,22,2,7,5,1,0,8,1,11
2,9,1,41,14,10,0,5,1,2,17
3,1,0,5,75,6,0,6,0,4,3
4,1,5,2,4,55,4,0,15,1,13
5,4,1,2,6,0,28,17,0,14,35
6,0,0,4,5,0,0,25,4,0,0
7,9,0,2,0,27,33,0,12,0,37
8,0,0,4,24,0,0,0,0,137,1
9,6,1,14,21,0,4,0,1,4,49


### XGBoost with oversampled features


In [35]:
oversampled_pipe = make_oversampled_feature_pipeline()
oversampled_pipe

Use pipe.set_params(over_sampler=.., estimator=..) to finish the pipeline


Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x28019cee0>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler', None),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x148b6d820>,
                                     kw_args={'fmax': None, 'fmin': 0.0,
                                              'hop_length': 128, 'n_fft': 512,
                                              'n_mels': 100, 'n_mfcc': 20,
                                              'sr': 16000})),
                ('scaler', StandardScaler()), ('estimator', None)])

In [7]:
oversampled_pipe.get_params()

{'memory': None,
 'steps': [('pad',
   FunctionTransformer(func=<function np_pad_wrapper at 0x280178d30>,
                       kw_args={'max_len': 64000})),
  ('over_sampler', None),
  ('mean_mfcc',
   FunctionTransformer(func=<function samples_to_mean_mfcc at 0x105ebc670>,
                       kw_args={'fmax': None, 'fmin': 0.0, 'hop_length': 128,
                                'n_fft': 512, 'n_mels': 100, 'n_mfcc': 20,
                                'sr': 16000})),
  ('scaler', StandardScaler()),
  ('estimator', None)],
 'verbose': False,
 'pad': FunctionTransformer(func=<function np_pad_wrapper at 0x280178d30>,
                     kw_args={'max_len': 64000}),
 'over_sampler': None,
 'mean_mfcc': FunctionTransformer(func=<function samples_to_mean_mfcc at 0x105ebc670>,
                     kw_args={'fmax': None, 'fmin': 0.0, 'hop_length': 128,
                              'n_fft': 512, 'n_mels': 100, 'n_mfcc': 20,
                              'sr': 16000}),
 'scaler': Standar

In [38]:
oversampled_pipe.set_params(estimator=XGBClassifier(n_estimators=400, 
                              learning_rate=0.025, 
                              max_depth=8, 
                              objective='multi:softmax',
                              nthread=4),                          
                           mean_mfcc__kw_args={'fmin':0, 'fmax':3000, 'n_mels': 98},
                           over_sampler=OverSampler())

Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x28019cee0>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x148b6d820>,
                                     kw_args={'fmax': 3000, 'fmin': 0,
                                              'n_mels': 98})),
                ('scaler', StandardScaler()),
                ('esti...
                               gamma=None, gpu_id=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.025, max_bin=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=8, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=400,
                               n_jobs=None, nthread=4, num_parallel_tree=None,
                               objective='multi:softmax', predictor=None,
                               random_state=None, ...))])

In [39]:
%%time
oversampled_pipe.fit(train_fold, train_y);

CPU times: user 8min 48s, sys: 55.9 s, total: 9min 44s
Wall time: 2min 12s


Pipeline(steps=[('pad',
                 FunctionTransformer(func=<function np_pad_wrapper at 0x28019cee0>,
                                     kw_args={'max_len': 64000})),
                ('over_sampler',
                 AugmentingRandomOversampler(sampling_strategy='minority')),
                ('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x148b6d820>,
                                     kw_args={'fmax': 3000, 'fmin': 0,
                                              'n_mels': 98})),
                ('scaler', StandardScaler()),
                ('esti...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.025, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=8, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=400, n_jobs=4, nthread=4,
                               num_parallel_tree=1, objective='multi:softmax',
                               predictor='auto', random_state=0, ...))])

In [42]:
xgb_ovsamp_val_pred = oversampled_pipe.predict(val_fold)
print(classification_report(
    combine_classes([0,2,4,5,6,7,9], val_y),#[1,3, 8]
    combine_classes([0,2,4,5,6,7,9], xgb_ovsamp_val_pred)#
))
print(classification_report(
     val_y, 
    xgb_ovsamp_val_pred))
labels = data.metadata[['classID', 'class']].drop_duplicates().sort_values('classID')['class'].to_list()
pd.DataFrame(confusion_matrix(val_y, xgb_ovsamp_val_pred, ),
             index=labels,#[0,1,2,3,4,5,6,7,8, 9], 
             columns=labels)#[0,1,2,3,4,5,6,7,8, 9])

              precision    recall  f1-score   support

           0       0.56      0.48      0.52       100
           2       0.47      0.47      0.47       100
           4       0.42      0.56      0.48       100
           5       0.32      0.27      0.29       107
           6       0.92      0.63      0.75        38
           7       0.21      0.08      0.12       120
           8       0.80      0.83      0.82       325
           9       0.31      0.54      0.40       100

    accuracy                           0.54       990
   macro avg       0.50      0.48      0.48       990
weighted avg       0.54      0.54      0.53       990

              precision    recall  f1-score   support

           0       0.56      0.48      0.52       100
           1       0.64      0.39      0.48        59
           2       0.47      0.47      0.47       100
           3       0.57      0.75      0.65       100
           4       0.42      0.56      0.48       100
           5       0.32 

,air_conditioner,car_horn,children_playing,dog_bark,drilling,engine_idling,gun_shot,jackhammer,siren,street_music
air_conditioner,48,1,11,0,15,10,0,0,1,14
car_horn,1,23,0,8,5,2,0,7,2,11
children_playing,7,0,47,8,22,0,0,4,0,12
dog_bark,2,4,5,75,7,0,1,0,4,2
drilling,4,0,0,5,56,5,0,21,0,9
engine_idling,12,0,16,2,0,29,0,0,16,32
gun_shot,0,1,2,5,0,0,24,4,0,2
jackhammer,8,6,0,0,26,37,1,10,0,32
siren,0,0,4,13,2,3,0,0,140,4
street_music,4,1,15,15,1,4,0,1,5,54


#### oversampled gridsearch

In [85]:
mean_mfcc_grid = []
for n_mfcc in [13, 16, 20]:#range(13, 26):
    for n_fft in range(256, 1025, 256):
        for fmin in [0.0, 300.0, 500.0]: #50.0, 100.0, 200.0, 300.0, 400.0, 500.0]:
            for fmax in [None, 3000.0, 4000.0, 5000.0]: #, 4000.0, 3000.0]:
                mean_mfcc_grid.append({'sr':16000, 'n_mfcc': n_mfcc, 'n_fft': n_fft,
                                       'hop_length': 128, 'fmin': fmin, 'fmax': fmax})
len(mean_mfcc_grid)

144

In [86]:
xgb_grid = [
    XGBClassifier(n_estimators=400, learning_rate=0.05, max_depth=8, objective='multi:softmax'),
    XGBClassifier(n_estimators=400, learning_rate=0.025, max_depth=8, objective='multi:softmax')
]

In [87]:
from sklearn.model_selection import GridSearchCV

In [91]:
%%time
gs = GridSearchCV(oversampled_pipe, 
                  dict(mean_mfcc__kw_args=mean_mfcc_grid,
                      estimator=xgb_grid), 
                  cv=2, verbose=2, n_jobs=1)
gs_res = gs.fit(train_fold, train_y)

Fitting 2 folds for each of 288 candidates, totalling 576 fits


/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


[CV] END estimator=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', predictor=None,
              random_state=None, reg_alpha=None, ...), mean_mfcc__kw_args={'sr': 16000, 'n_mfcc': 13, 'n_fft': 256, 'hop_length': 128, 'fmin': 0.0, 'fmax': None}; total time=  51.1s


/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


[CV] END estimator=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', predictor=None,
              random_state=None, reg_alpha=None, ...), mean_mfcc__kw_args={'sr': 16000, 'n_mfcc': 13, 'n_fft': 256, 'hop_length': 128, 'fmin': 0.0, 'fmax': None}; total time=  50.5s


...

[CV] END estimator=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.025, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', predictor=None,
              random_state=None, reg_alpha=None, ...), mean_mfcc__kw_args={'sr': 16000, 'n_mfcc': 20, 'n_fft': 1024, 'hop_length': 128, 'fmin': 500.0, 'fmax': 5000.0}; total time= 1.2min


/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=960
  return f(*args, **kwargs)
/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=800
  return f(*args, **kwargs)
/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=960
  return f(*args, **kwargs)


[CV] END estimator=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.025, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', predictor=None,
              random_state=None, reg_alpha=None, ...), mean_mfcc__kw_args={'sr': 16000, 'n_mfcc': 20, 'n_fft': 1024, 'hop_length': 128, 'fmin': 500.0, 'fmax': 5000.0}; total time= 1.3min


/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


CPU times: user 2d 7h 28min 45s, sys: 13h 3min 21s, total: 2d 20h 32min 7s
Wall time: 9h 9min 21s


In [93]:
gs_res.best_params_

{'estimator': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, gamma=None,
               gpu_id=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.05, max_bin=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None, n_estimators=400, n_jobs=None,
               num_parallel_tree=None, objective='multi:softmax', predictor=None,
               random_state=None, reg_alpha=None, ...),
 'mean_mfcc__kw_args': {'sr': 16000,
  'n_mfcc': 16,
  'n_fft': 256,
  'hop_length': 128,
  'fmin': 0.0,
  'fmax': 3000.0}}

In [94]:
xgb_gs_val_pred = gs.best_estimator_.predict(val_fold)
print(classification_report(
    combine_classes([1, 8], val_y),
    combine_classes([1, 8], xgb_gs_val_pred)
))
print(classification_report(
     val_y, 
    xgb_gs_val_pred))
pd.DataFrame(confusion_matrix(val_y, xgb_gs_val_pred),
             index=[0,1,2,3,4,5,6,7,8,9], 
             columns=[0,1,2,3,4,5,6,7,8,9])

/Users/axel/.pyenv/versions/3.8.12/envs/us8kdata/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


              precision    recall  f1-score   support

           1       0.39      0.53      0.45        59
           7       0.92      0.91      0.91       765
           8       0.84      0.79      0.81       166

    accuracy                           0.86       990
   macro avg       0.72      0.74      0.72       990
weighted avg       0.87      0.86      0.87       990

              precision    recall  f1-score   support

           0       0.47      0.20      0.28       100
           1       0.39      0.53      0.45        59
           2       0.40      0.49      0.44       100
           3       0.50      0.77      0.60       100
           4       0.63      0.54      0.58       100
           5       0.37      0.24      0.29       107
           6       0.75      0.63      0.69        38
           7       0.24      0.07      0.11       120
           8       0.84      0.79      0.81       166
           9       0.24      0.49      0.32       100

    accuracy           

,0,1,2,3,4,5,6,7,8,9
0,20,10,32,0,13,4,0,0,0,21
1,2,31,1,5,8,1,0,3,1,7
2,4,0,49,17,3,3,5,1,0,18
3,0,0,7,77,2,0,3,0,6,5
4,2,7,1,5,54,1,0,20,0,10
5,6,1,9,2,0,26,0,0,12,51
6,0,0,5,4,0,0,24,3,0,2
7,4,24,4,0,5,30,0,9,0,44
8,0,0,5,30,0,0,0,0,131,0
9,5,6,11,15,1,6,0,1,6,49


In [19]:
%%time
post_search_pipeline = make_oversampled_feature_pipeline()
post_search_pipeline.set_params(
**{'estimator': XGBClassifier(learning_rate=0.05, max_depth=8,
               n_estimators=400, objective='multi:softmax'),
 'mean_mfcc__kw_args': {'sr': 16000,'n_mfcc': 16,
  'n_fft': 256,'hop_length': 128,
  'fmin': 0.0, 'fmax': 3000.0, 'n_mels': 98},
  'over_sampler': OverSampler(augment_method='AddGaussianNoise')}
)
post_search_pipeline.fit(train_fold, train_y)

Use pipe.set_params(over_sampler=.., estimator=..) to finish the pipeline


KeyboardInterrupt: 

In [17]:
xgb_ps_val_pred = post_search_pipeline.predict(val_fold)
print(classification_report(
    combine_classes([1, 8], val_y),
    combine_classes([1, 8], xgb_ps_val_pred)
))
print(classification_report(
     val_y, 
    xgb_ps_val_pred))
pd.DataFrame(confusion_matrix(val_y, xgb_ps_val_pred),
             index=[0,1,2,3,4,5,6,7,8,9], 
             columns=[0,1,2,3,4,5,6,7,8,9])

              precision    recall  f1-score   support

           1       0.36      0.44      0.40        59
           7       0.91      0.89      0.90       765
           8       0.78      0.79      0.78       166

    accuracy                           0.85       990
   macro avg       0.68      0.71      0.69       990
weighted avg       0.86      0.85      0.85       990

              precision    recall  f1-score   support

           0       0.19      0.08      0.11       100
           1       0.36      0.44      0.40        59
           2       0.49      0.43      0.46       100
           3       0.43      0.77      0.55       100
           4       0.61      0.56      0.58       100
           5       0.42      0.25      0.31       107
           6       0.55      0.61      0.57        38
           7       0.24      0.08      0.12       120
           8       0.78      0.79      0.78       166
           9       0.23      0.45      0.30       100

    accuracy           

,0,1,2,3,4,5,6,7,8,9
0,8,16,6,7,4,5,0,0,23,31
1,3,26,0,9,8,1,0,5,1,6
2,5,0,43,19,8,1,5,2,0,17
3,2,1,7,77,2,0,2,0,5,4
4,2,7,3,1,56,3,0,19,0,9
5,6,0,5,14,0,27,11,0,5,39
6,0,0,6,4,0,0,23,4,0,1
7,11,14,2,0,13,24,0,10,0,46
8,0,0,3,31,0,0,0,0,131,1
9,5,8,12,18,1,4,1,2,4,45


## VGGish

# Sequence of samples classification
Model architectures for treating each file as a sequence of sample frames, making the classification either based on a subset or on the frame as a whole.

## HMM

## RNN